## Setup

### Imports

In [7]:
#importing modules
import os, sys, json, datetime, re, xlrd  # Provides OS-dependent functionality, system-specific parameters, JSON handling, and date/time manipulation
import pandas as pd             # Provides data structures and data analysis tools
from openpyxl import Workbook
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import glob
import time
from tqdm import tqdm
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'
from IPython.display import display_markdown

import sys
print(sys.executable)

from setuptools import find_packages
print(find_packages())


from cprl_functions.state_capture import thi_states,state_ref, state_coding, state_pat, state_abv_pat
from cprl_functions.text_printing import bordered
from cprl_functions.defined_functions import create_pk

c:\Users\clutz\hunt_env\Scripts\python.exe
[]


## Data Gathering

### gathering leg files

In [15]:
#gather all legislator files from done folder
#committee data should be updated before pulling this

# os.chdir(r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\done')
os.chdir(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\2025')
legislator_files = glob.glob('**/*_info*.xlsx')
leg_files = [f for f in legislator_files if not f.endswith('_old.xlsx')]
leg_files = [f for f in leg_files if re.search(r'12_11', str(f))]

print(leg_files)


['AL\\AL_legislators_info_2024_12_11.xlsx', 'CT\\CT_legislators_info_2024_12_11.xlsx', 'IL\\IL_legislators_info_2024_12_11.xlsx', 'IN\\IN_legislators_info_2024_12_11.xlsx', 'KS\\KS_legislators_info_2024_12_11.xlsx', 'MO\\MO_legislators_info_2024_12_11.xlsx', 'NC\\NC_legislators_info_2024_12_11.xlsx', 'ND\\ND_legislators_info_2024_12_11.xlsx', 'NM\\NM_legislators_info_2024_12_11.xlsx', 'OH\\OH_legislators_info_2024_12_11.xlsx', 'OK\\OK_legislators_info_2024_12_11.xlsx', 'VA\\VA_legislators_info_2024_12_11.xlsx', 'WV\\WV_legislators_info_2024_12_11.xlsx']


In [ ]:
# #gathering legislator files from one drive 
# os.chdir(r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\done')
# legislator_files = glob.glob('*.xlsx') 

# for i,file in enumerate(legislator_files):
#     if '_legislators' not in str(file).lower():
#         print("deleting: " + str(legislator_files[i]))
#         del legislator_files[i] 

In [17]:

#compiles legislator files into one file
dfs = []
for i,file in enumerate(legislator_files):
    # print('working on file:' + str(file))
    sheets_dict = pd.read_excel(file, engine="openpyxl", sheet_name=None)
    sheet_names = list(sheets_dict.keys())
    for s in sheet_names:
        df = pd.read_excel(file, engine="openpyxl", sheet_name=s)
        df = df.iloc[:,:9]
        dfs.append(df)

    df = pd.concat(dfs)

all_legs = pd.concat(dfs)

### gathering attendance data

In [ ]:
# import all attendance data files
os.chdir(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\attendance data')
events = glob.glob("*.xlsx")
print(*events, sep='\n')

## Cleaning and Field Creation

### Filling in State info

looks for state names in the title, org, and state fields

In [19]:

test_state = "North Carolina"
re.findall(state_pat, test_state)
print(events)

['DE_LEG_ED_Dinner_2023.xlsx', 'ECLS_2024.xlsx', 'ElevateNC_C4_M3.xlsx', 'ElevateNC_C4_M4.xlsx', 'HBCU_Caucus_2024.xlsx', 'HKF_C10_S1.xlsx', 'HKF_Regional_Visit_FAU.xlsx', 'HSPF_C4_M1.xlsx', 'HSPF_C4_M2.xlsx', 'HSPF_C4_M3.xlsx', 'MO_SLR_2023.xlsx', 'NCCCS_M4.xlsx', 'NC_EC_Roundtable_2024.xlsx', 'NC_HLR_2024.xlsx', 'ND_Literacy_taskforce_2024.xlsx', 'ND_SLR_2023.xlsx', 'ND_SLR_2024.xlsx', 'ND_TRR_M1.xlsx', 'ND_TRR_m2.xlsx', 'ND_TRR_m3.xlsx', 'OH_SLR_2023.xlsx', 'OH_SLR_2024.xlsx', 'OK_SLR_2023.xlsx', 'OK_SLR_2024.xlsx', 'SC_Leg_Ed_Dinner_2023.xlsx', 'The_Path_Forward_2024.xlsx', 'WV_SLR_2023.xlsx', 'WV_SLR_2024.xlsx']


In [20]:
dfs = []
vals_changed = 0
for event in events:
    df = pd.read_excel(event)
    # print('######################')
    # print(bordered(event))
    
    # print(*df.columns)
    event_name = str(event).split('.')[0].strip().replace('_', ' ')
    df = df.iloc[:,:8]
    df.loc[:,'event name'] = event_name
    
    break_all = False
    # #print(df)
    # continue
    # display_markdown(f'## {event_name}', raw=True)
    for i,state in enumerate(df['state']):
        
        # if "HKF" in str(df['event name'].iloc[i]): 
        #     print('###############################')
        #     print(f'first name: {str(df.loc[i,'first_name'])}')
        #     print(f'last name: {str(df.loc[i,'last_name'])}')
        #     print(f'role: {str(df.loc[i,'role'])}')
        #     print(f'org: {str(df.loc[i,'org'])}')
        #     print(f'title: {str(df.loc[i,'title'])}')
        
        
        #     hkf_match = re.findall(state_pat, str(df['org'].iloc[i]))
        #     print(hkf_match)
        #     if len(hkf_match) == 0:
                
        #     else:
        #         print(hkf_match[0])
        #         continue

        
        testing_string = str(df['title'].iloc[i]) + " " + str(df['org'].iloc[i])
        # #print(testing_string)
        testing_string = testing_string.lstrip('nan').lstrip().strip()
        # #print(re.match(r'[Rr]epresentative|[Ss]enator|[Ll]egislator|[Ss]enate|[Hh]ouse of ([Rr]epresentatives)?(Delegates)?|[Dd]istrict|[Ss]tate [Hh]ouse',str(testing_string)))
        # continue
        # #print('###########')
        # #print(df.loc[i, list(df.columns[:5]) + [df.columns[-1]]])
        # #print('\n')

        # if 'HFK' in str(df.loc[i,'role']):
        # try:
        state_match_uc = re.findall(state_pat, str(df.loc[i,'org']))
        state_match = [x for x in state_match_uc if len(x) > 0]



        # if 'hkf' in str(df.loc[i,'role']).lower():
        #     print('HKF')
        #     print(f'first name: {str(df.loc[i,'first_name'])}')
        #     print(f'last name: {str(df.loc[i,'last_name'])}')
        #     print(f'role: {str(df.loc[i,'role'])}')
        #     print(f'org: {str(df.loc[i,'org'])}')
        #     print(f'title: {str(df.loc[i,'title'])}')
        #     try:
        #         print(f'state match: {state_match[0]}')
        #         continue
        #     except:
        #         print('no match found')
        # else:
        #     continue
            

        
       

        # First match test
        if len(state_match) == 0:
            ##print('no regular state match')
            ##print(state_match_uc)
            state_abv_match_uc = re.findall(state_abv_pat, str(df['org'].iloc[i]))
            state_abv_match = [x for x in state_abv_match_uc if len(x) > 0]
            # Second match test
            if len(state_abv_match) == 0:
                ##print('no state abbreviation match')
                ##print(state_abv_match_uc)
                state_abv_event_match_uc = re.findall(state_abv_pat, str(df['event name'].iloc[i]))
                state_abv_event_match = [x for x in state_abv_event_match_uc if len(x) > 0]
                # Third match test
                if len(state_abv_event_match) == 0:
                    #print('no state abv event match')
                    #print(state_abv_event_match_uc)
                    #print(f'first name: {str(df.loc[i,'first_name'])}')
                    #print(f'last name: {str(df.loc[i,'last_name'])}')
                    #print(f'role: {str(df.loc[i,'role'])}')
                    #print(f'org: {str(df.loc[i,'org'])}')
                    #print(f'title: {str(df.loc[i,'title'])}')
                    continue
                elif len(state_abv_event_match) > 1:
                    #print('more than one match?')
                    # break_all = True
                    break
                else:
                    ##print("abv in event match")
                    state_val = str(state_abv_event_match[0])
                    # df.loc[i,'state'] = None
                    df.loc[i,'state'] = str(df.loc[i,'state'])
                    df.loc[i,'state'] = state_val
                    ##print(state_val)
                    vals_changed += 1
            elif len(state_abv_match) > 1:
                #print('more than one match?')
                #print(state_abv_match)
                ##print(df.loc[i, list(df.columns[:5]) + [df.columns[-1]]])
                break_all = True
                break
            else:
                #print("regular abreviation match")
                
                state_val = str(state_abv_match[0])
                # df.loc[i,'state'] = None
                df.loc[i,'state'] = str(df.loc[i,'state'])
                df.loc[i,'state'] = state_val
                ##print(state_val)
                vals_changed += 1

            # ##print('###########')
            # ##print(df.loc[i, list(df.columns[:5]) + [df.columns[-1]]])
            # ##print('\n')
            # break
        elif len(state_match) > 1:
            #print("more than one match?")
            # break_all = True
            break
        else:
            # #print("normal state match")
            state_val_dirty = str(state_match[0])
            state_val = state_ref.get(state_val_dirty)
            # df.loc[i,'state'] = str(df.loc[i,'state'])
            # df.loc[i,'state'] = None
            df.loc[i,'state'] = str(state_val)
            #print(state_val)
            vals_changed += 1
        # else:
        #     # ##print('#########################')
        #     # ##print('NOT A REP OR SEN')
        #     # ##print(df.loc[i,['first_name','last_name','title', 'org']])
        #     continue
            # ##print(df.loc[i, list(df.columns[3:5]) + [df.columns[-1]]])
            # ##print('\n')
    # if break_all == True:
        # break
    

    dfs.append(df)
event_data = pd.concat(dfs)
event_data.reset_index(inplace=True, drop = True)


event_data

,first_name,last_name,honorific,title,org,district,role,state,event name
0,Kyra,Hoffner,Senator,District 014 Senator,Deleware Senate,NaN,NaN,DE,DE LEG ED Dinner 2023
1,Russell,Huxtable,Senator,District 006 Senator,Deleware Senate,NaN,NaN,DE,DE LEG ED Dinner 2023
2,Laura,Sturgeon,Senator,District 004 Senator,Deleware Senate,NaN,NaN,DE,DE LEG ED Dinner 2023
3,Jeff,Hilovsky,Representative,District 004 Representative,Deleware House of Representatives,NaN,NaN,DE,DE LEG ED Dinner 2023
4,Sophie,Phillips,Representative,District 018 Representative,Deleware House of Representatives,NaN,NaN,DE,DE LEG ED Dinner 2023
...,...,...,...,...,...,...,...,...,...
1417,Hank,Hager,NaN,Chief Counsel,West Virginia State Senate Education Committee,NaN,Invited Guests,WV,WV SLR 2024
1418,Jeff,Kelley,NaN,Assistant Superintendent of District & School ...,West Virginia Department of Education,NaN,Invited Guests,WV,WV SLR 2024
1419,JB,McCuskey,Auditor,West Virginia State Auditor,West Virginia State Auditor's Office,NaN,Invited Guests,WV,WV SLR 2024
1420,Mike,Queen,NaN,Deputy Secretary of State,West Virginia Secretary of State's Office,NaN,Invited Guests,WV,WV SLR 2024


In [21]:

#looks for state names and replaces them with state initials
for i,j in enumerate(event_data['state']):
    
    if isinstance(j, float):
        continue
    elif re.search(r'[A-Z]{2}', str(j)):
        continue
    else:
        val = state_ref.get(str(j))
        event_data.loc[i,'state'] = str(val)

In [22]:
os.chdir(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\attendance data\exports')
event_data.to_csv("event_data_export_12_11_2024.csv", index=False)

### District Cleaning

In [23]:
#Legislator search pattern set up
title_pattern = r'[Rr]epresentative|[Ss]enator|[Ll]egislator'
org_pattern = r'[Ss]enate|[Hh]ouse of ([Rr]epresentatives)?(Delegates)?|(?<!School )(?:House District|District)|[Ss]tate [Hh]ouse'
exclude_pattern = r'[Aa]id(e)?|[Aa]ssistant|[Ss]taff'

In [24]:
# Filter for state legislators
filtered_df = event_data[
    (
        event_data['title'].astype(str).apply(lambda x: bool(re.search(title_pattern, x))) |
        event_data['org'].astype(str).apply(lambda x: bool(re.search(org_pattern, x)))
    ) &
    ~(
        event_data['org'].astype(str).apply(lambda x: bool(re.search(exclude_pattern, x)))
    )
]

In [ ]:

#regine values with districts
w_districts = filtered_df[(filtered_df['org'].str.contains(r'[Dd]istrict\s?\d{1,3}[A-Za-z]?|[Dd](-|\s)?\d{2,3}[A-Za-z]?', regex=True) | 
                filtered_df['title'].str.contains(r'[Dd]istrict\s?\d{1,3}[A-Za-z]?|[Dd](-|\s)?\d{2,3}[A-Za-z]?', regex=True)) &
                ~(filtered_df['title'].str.contains(r'[Aa]ssistant|[Aa]id(e)?|[Ss]taff', regex=True, na=False))]

#find no districts
n_districts = filtered_df[~(filtered_df['org'].str.contains(r'[Dd]istrict\s?\d{1,3}[A-Za-z]?|[Dd](-|\s)?\d{2,3}[A-Za-z]?', regex=True) | 
                filtered_df['title'].str.contains(r'[Dd]istrict\s?\d{1,3}[A-Za-z]?|[Dd](-|\s)?\d{2,3}[A-Za-z]?', regex=True)) &
                ~(filtered_df['title'].str.contains(r'[Aa]ssistant|[Aa]id(e)?|[Ss]taff', regex=True, na=False))]


#clean up
w_districts.reset_index(inplace=True, drop=True)


In [26]:
#Looking for districts in title and org field
i = 0
for a,b in zip(w_districts.title, w_districts.org):


    has_a = False
    has_b = False
    if 'district' in str(a).lower() or re.search(r'[Dd]-?\s?\d{1,3}[A-Za-z]?', str(a)):
        match_a = re.findall(r'[Dd]istrict\s?\d{1,3}[A-Za-z]?|[Dd]-?\s?\d{1,3}[A-Za-z]?', str(a))
        match_a = [x for x in match_a if len(x) > 0]
        if len(match_a) == 0:
            print('no results for title')
            print(a)
            
        else:
            has_a = True
            match = re.sub(r'[A-Za-z]','',str(match_a[0]))

    
    if 'district' in str(b).lower() or re.search(r'[Dd]-?\s?\d{1,3}[A-Za-z]?', str(b)):
        match_b = re.findall(r'[Dd]istrict\s?\d{1,3}[A-Za-z]?|[Dd]-?\s?\d{1,3}[A-Za-z]?', str(b))
        match_b = [x for x in match_b if len(x) > 0]
        if len(match_b) == 0:
            print('no results for org')
            print(b)
        
        else:
            has_b = True
            match = re.sub(r'[A-Za-z]','',str(match_b[0]))


    if has_b == True or has_a == True:
        match_final = re.findall(r'\d+[A-Za-z]?', str(match))
        # print("final match: " + str(match_final[0]))
        # print("putting it on row: " + str(i))
        w_districts.loc[i, 'district'] = str(match_final[0]).strip().lstrip('0')

    i +=1
# w_districts

In [71]:
#Data export to create patch file
# os.chdir(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\attendance data\exports')
# n_districts.to_csv('no_districts.csv', index=False)

In [27]:
# Process patch file 
patch_file = r"C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\attendance data\patch files\no_districts_patch.csv"
districts_patch = pd.read_csv(patch_file)

#pull together all data
patched_df = pd.concat([w_districts,districts_patch])
patched_df.reset_index(inplace=True, drop=True)


#looks through and assigns chamber column to either house or senate
w_districts['chamber'] = ""
i = 0
for a,b in zip(patched_df.title, patched_df.org):
    if re.search(r'[Hh]ouse|[Ss]enate', str(b)):
        if re.search(r'[Hh]ouse', str(b)):
            chamber = "House"
        elif re.search(r'[Ss]enate', str(b)):
            chamber = "Senate"
    elif re.search(r'[Rr]epresentative|[Ss]enator|[Dd]elegate', str(a)):
        if re.search(r'[Rr]epresentative|[Dd]elegate', str(a)):
            chamber = "House"
        elif re.search(r'[Ss]enator', str(a)):
            chamber = "Senate"

    try:
        patched_df.loc[i,'chamber'] = str(chamber)
        i += 1
    except:
        i += 1
        continue


#find only thi states
patched_df = patched_df[patched_df['state'].isin(thi_states)]
patched_df = patched_df[~patched_df['district'].isna()]
patched_df.reset_index(inplace=True, drop=True)

#name edits
patched_df['first_name'] = patched_df['first_name'].str.strip().str.title()
patched_df['last_name'] = patched_df['last_name'].str.strip().str.title().str.replace("' ", "'")
# condition = patched_df['last_name'].str.contains(r'(?!\w+)\s(?<!\w)', regex = True)
# patched_df.loc[condition, 'last_name'] = (patched_df['last_name'].str.split(r'\s*,\s*(?=[A-Z])').str[0])
# Identify rows where 'last_name' has two words separated by whitespace

# patched_df

In [28]:
cleaned_df,duplicates = create_pk(patched_df,'district', 'chamber')

# clean_dfs = [cleaned_df,duplicates]
# clean_df = pd.concat(clean_dfs)


In [29]:
# print(duplicates.columns)
#grouping data together and getting list of events per legislator
grouped_df = duplicates.groupby(['primary_key','last_name']).agg({
    'state': 'first',
    'first_name': 'first',
    'event name': lambda x: '|'.join(
        list(set(f"{sc} ({ac})" if not pd.isna(ac) else f"{sc}"
        for sc, ac in zip(duplicates.loc[x.index, 'event name'], duplicates.loc[x.index, 'role']))))

}).reset_index()
# grouped_df.reset_index()
grouped_df.rename(columns={'event name': 'events'}, inplace=True)
# state_coding
# grouped_df
# print(cleaned_df.columns)


#clean up grouped data
non_dupe_df = cleaned_df.loc[:,['primary_key', 'first_name','last_name', 'state','event name']]
non_dupe_df.rename(columns={'event name': 'events'}, inplace=True)

#pull event data back together
merged_dfs = [grouped_df, non_dupe_df]
leg_events_df = pd.concat(merged_dfs)
# print(leg_events_df.columns)


#find dupes
leg_events_df_dupes = leg_events_df[leg_events_df.duplicated(subset='primary_key', keep = False)]
leg_events_df_dupes.reset_index(inplace = True, drop = True)


#find none dupes and add seat
leg_events_df_nodupes = leg_events_df[~leg_events_df.duplicated(subset='primary_key', keep = False)]
leg_events_df_nodupes['seat'] = '00'



In [34]:
#pull in key lookup
os.chdir(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\connectors')

leg_lookup_file = r"C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\all_legs_files\all_legs_files_2024_12_11.csv"
leg_lookup_ref = pd.read_csv(leg_lookup_file)

#groupby data to get primary key and last names associated with it
loop_group = leg_lookup_ref.groupby(['primary_key']).agg({
    'Last Name': lambda x: '|'.join(
        list(set(f"{sc}" if not pd.isna(sc) else "not found"
        for sc in leg_lookup_ref.loc[x.index, 'Last Name'])))
}).reset_index()

#set up dict for lookup
loop_dict = loop_group.set_index('primary_key')['Last Name'].to_dict()

#create seat for dupes
leg_events_df_dupes['seat'] = np.nan
for i,j in enumerate(leg_events_df_dupes['primary_key']):
    #ensure j is an integer
    j = int(j)
    
    #get name we are looking for
    name_to_check = leg_events_df_dupes.loc[i,'last_name']
    trunc_name = str(name_to_check)[:3]
    # print(f"checking for {trunc_name}")
    
    #look for the key, get results, and split up the names into a list
    results = loop_dict.get(j)
    names = results.split('|')
    
    #get length of names
    n = len(names)-1
    
    #go through and check if the name matches either of the one in the list and return the seat
    for ik,name in enumerate(names):
        # print(f"going through {ik}")
        # print(type(name))
        # print(type(trunc_name))
        if re.search(f'^{trunc_name.strip()}', str(name)):
            # print('found it')
            seat = ik + 1
            leg_events_df_dupes.loc[i,'seat'] = f'0{seat}'
            break
        #stops if we are on the last iteration and still no seat
        elif ik == n:
            print(f"couldn't find one for '{trunc_name}' aka '{name_to_check}' in {names} with length for it being {len(trunc_name)}")



AttributeError: 'NoneType' object has no attribute 'split'

In [31]:
# print(leg_events_df_nodupes.columns)
# print(leg_events_df_dupes.columns)

#pull all event data back together
events_dfs = [leg_events_df_dupes, leg_events_df_nodupes]
events_df = pd.concat(events_dfs)
events_df.dropna(subset=['seat'], inplace=True)
events_df.reset_index(inplace=True, drop=True)
# events_df


Having issues where just applying the last part of the create pk function just assigns people with same last name different seats. Might need to run create_pk without athe seat numbers first, group the (but keeping the last name for differences) and then assign seat numbers based on that. Can create a mid point look up value

## Scoring

Cell below calculates the activities score from the attendance data

In [32]:
#Calculating Score for loop


# print(events_df.columns)
#For loop description: goes through events column and gathers information for activities scoring
events_df.loc[:, 'activities_score'] = 0
for i,j in enumerate(events_df['events']):
    
    
    # split up events
    event_split = str(j).split('|')
    events = ";".join(event_split)
    # if len(event_split) < 2:
    #     continue


    #compile name for print statements

    # fname = grouped_df.at[i,'first_name']
    lname = events_df.at[i,'last_name']

    # names = [fname, lname]
    # name = " ".join(names)
    
    
    # display_markdown(f' ## {name}', raw=True)
    # print(bordered(events))

    #For loop description: go through each event and score 
    scores = []
    for event in event_split:

        #intializing boolean values for scoring    
        score = 0
        speaker = False
        is_hkf = False
        dev_program = False
        in_state = False
        out_state = False
        is_slr = False
        dinner_or_lunch = False
        # non_slr = False
        speaker = False
        # #print('#################')
        # #print(*grouped_df.loc[i,['helper','first_name', 'last_name', 'events']], sep=" \ ")
        
        
        # #print(bordered(event))

        #Look through for roles in events
        if re.search(r'\(.+\)', str(event)):
            match = re.findall(r'\(.+\)', str(event))
            match_refine = [x for x in match if len(x) != 0]
            #print("match refine results", match_refine)
            if len(match_refine) != 0:
                for m in match_refine:
                    if re.search('speaker|presenter', str(m).lower()):
                        # print('found a speaker')
                        speaker = True
                    elif 'HKF' in str(m):
                        #print('THERE IS HKF IN THE RESULTS')
                        is_hkf = True
                
        
        #is it just a short engagment such as a dinner or lunch?
        if re.search(r'[Dd]inner|[Ll]unch', str(event)):
            dinner_or_lunch = True

        #get state
        state = events_df.loc[i,'state']
        
        #looking for whether events where in state or out of state
        if 'ECLS' not in str(event) or "HKF" not in str(event):
            #print("no ecls or hKF")
            try:
                event_state = re.findall(state_abv_pat, str(event))[0].strip()
                if event_state == state:
                #print("states match")
                    in_state = True
                else:
                    out_state = True
            except:
                out_state = True
                # print(str(event))
                # print('no state match')
        else:
            out_state = True
            
            
        #lower dev program?
        if 'HSPF' in str(event) or 'Elevate' in str(event):
            dev_program = True

        #State Legislator event?
        if re.search(r'SLR|HLR',str(event)):
            is_slr = True


        # if re.search(r'\s[Mm]\d', str(event)):
        #     non_slr = True
        
        variables = [
        speaker,
        is_hkf,
        dev_program,
        in_state,
        out_state,
        is_slr,
        dinner_or_lunch
        ]

        #Trouble shooting print statement to make sure logic is working
        # #print('quick look at logic')
        # for var_name, var_value in zip(['speaker', 'is_hfk', 'dev_program', 'in_state', 'out_state', 'is_slr', 'dinner_or_lunch', 'non_slr', 'out_of_state'], variables):
        #     #print(bordered(f"{var_name}: {var_value}"))
        

        #Event data scoring 
        if is_slr == True:
            score += 15
            # print(f'adding 15 for {name} due to being an slr')
        # else:
        #     score += 10
        #     #print(f'adding 10 for {name}')

        elif dev_program == True:
            score += 15
            # print(f'adding 15 for {name} due to being in an dev program')
        elif dinner_or_lunch == True:
            score += 5
            # print(f'adding 5 for {name} due to being a lunch or dinner')
        else:
            score += 10
            # print("adding 10 for full day event with no other attributes")

        
        #check for speaker
        if speaker == True:
            if in_state == True:
                score += 0
                #if in state no additional points
                # print(f'adding 0 for {name} for being in state speaker')
            elif out_state == True:
                #if out of state add 5 more points for speakers
                # print(f'adding 5 for {name} due ot being a speaker at an out of state event')
                score += 5
        
        # check for hkf
        if is_hkf == True:
            score += 20
            # print(f'adding 20 for {lname} due to being hkf')


        
        # print(bordered(score))
        scores.append(score)

    # display_markdown(f' ### {name}', raw=True)
    # print(scores)
    total = sum(scores)
    # print("total: ",total)
    
    
    events_df.loc[i, 'activities_score'] = total

In [ ]:
from datetime import date
#export activity scores df
activity_scores = events_df.loc[:,['primary_key','seat','first_name','last_name','activities_score', 'events']]

#make full_pk and convert to int
activity_scores['full_pk'] = activity_scores['primary_key'].astype(str) + activity_scores['seat'].astype(str)
activity_scores['full_pk'] = activity_scores['full_pk'].astype(int)

#put full_pk to front
activity_scores = activity_scores.drop('seat', axis=1)
first_column = activity_scores.pop('full_pk')
activity_scores.insert(0, 'full_pk', first_column)
print(*activity_scores.columns, sep = ' , ')
activity_scores

full_pk , primary_key , first_name , last_name , activities_score , events


In [ ]:



os.chdir(r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files')


activity_scores.to_csv(f'activity_scores{str(date.today()).replace('-','_')}.csv', index = False)



# activity_scores